# Imports

Importing all the libraries I might need for this notebook. 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # google API for geopraphical information 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

# Web scraping using Pandas


In [183]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
df=pd.read_html(url, header =0)
df

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [184]:
len(df)

3

This page contains 3 tables, I am going to need the first one only

In [8]:
df_Tto = df[0]

df_Tto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_Tto.shape

(180, 3)

Eliminating those records that have no Borough

In [10]:
df_Tto.drop(df_Tto[df_Tto.Borough == 'Not assigned'].index, inplace= True)
df_Tto.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Checking for not assigned Neighborhoods.

In [11]:
df_Tto[df_Tto.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


Chechinh for duplicated postal codes 

In [12]:
df_Tto[df_Tto.duplicated('Postal Code')]

,Postal Code,Borough,Neighborhood


In [13]:
df_Tto.shape

(103, 3)

# Getting coordinates for each record 

I tried two APIs to get Longitude and Latitude data. None of the worked. 

### Using Google / Geocoder

In [ ]:
address = '{}, Toronto , Ontario'.format(df_Tto.iloc[3,0])
address

##### This Code never worked, it enters in a loop and never got a value for latitude and longitude

        lat_lng_coords = None
        while(lat_lng_coords is None):
            g = geocoder.google(address)
            lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]

### Using geopy  

In [30]:
#using geopy to get the latitude and logitude of a city
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of M7A, Toronto , Ontario are 43.6534817, -79.3839347.


This API got better results than Geocoder / Google, but stil couldn't find many postal codes from the list 

### Using the provided CSV

In [36]:
coords_data = pd.read_csv('Geospatial_Coordinates.csv')

In [39]:
coords_data.set_index('Postal Code')

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


the new variable **df_tto** has the merge of the original set and the geo data from the CSV

In [61]:
df_tto=df_Tto.merge(coords_data, on='Postal Code')

In [71]:
df_tto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Analyzing Toronto

Finding the right lat and long to center the screen of my map

In [82]:
latitude = (df_tto['Latitude'].max()+df_tto['Latitude'].min())/2
longitude = (df_tto['Longitude'].max()+df_tto['Longitude'].min())/2

Creating a full map of all neighborhoods

In [83]:
# create map of Toronto using latitude and longitude values
map_tto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tto['Latitude'], df_tto['Longitude'], df_tto['Borough'], df_tto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tto)  
    
map_tto

### Connecting to FourSquare

In [84]:
CLIENT_ID = 'BI1YUQQYIU1XL3BSGMD1HRRQJKCW1V52LUUH3VL5HB0D2XGA' # your Foursquare ID
CLIENT_SECRET = 'SVWGLRACEUFZXTLQWDUJEN3YA40L3NCTL5OKRGGMHC0XOMYI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BI1YUQQYIU1XL3BSGMD1HRRQJKCW1V52LUUH3VL5HB0D2XGA
CLIENT_SECRET:SVWGLRACEUFZXTLQWDUJEN3YA40L3NCTL5OKRGGMHC0XOMYI


In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [86]:
tto_venues = getNearbyVenues(names=df_tto['Neighborhood'],
                                   latitudes=df_tto['Latitude'],
                                   longitudes=df_tto['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In the data there are 4 rows that make reference to the Neighborhoods. 
I will remove the category Neighborhood

In [107]:
tto_venues[tto_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
215,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
393,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
471,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
712,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


In [108]:
tto_venues.drop(tto_venues[tto_venues['Venue Category']=='Neighborhood'].index, inplace=True)

In [109]:
print(tto_venues.shape)
tto_venues.head(10)

(1329, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [110]:
tto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18


In [111]:
print('There are {} uniques categories.'.format(len(tto_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [113]:
# one hot encoding
tto_onehot = pd.get_dummies(tto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tto_onehot['Neighborhood'] = tto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tto_onehot.columns[-1]] + list(tto_onehot.columns[:-1])
tto_onehot = tto_onehot[fixed_columns]

tto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [114]:
tto_onehot.shape

(1329, 234)

In [115]:
tto_grouped = tto_onehot.groupby('Neighborhood').mean().reset_index()
tto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,

In [116]:
tto_grouped.shape

(96, 234)

In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [167]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tto_grouped['Neighborhood']

for ind in np.arange(tto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pub,Sandwich Place,Pharmacy,Gym,General Travel,Curling Ice,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Bridal Shop,Shopping Mall,Sandwich Place,Restaurant,Pizza Place,Mobile Phone Shop
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Yoga Studio,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Sushi Restaurant,Pharmacy,Thai Restaurant,Juice Bar,Butcher,Café


## Clustering

I decided to go with 4 clusters. 
5 clusters provided a cluster with just one point, and 4 or less was not granular enough. 

In [169]:
# set number of clusters
kclusters = 4

tto_grouped_clustering = tto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0,
       3, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 2, 0, 0, 0, 2,
       0, 0, 0, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3,
       2, 0, 0, 0, 3, 3, 0, 2])

In [170]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [171]:
tto_merged = df_tto

# merge toronto_grouped with df_tto to add latitude/longitude for each neighborhood
tto_merged = tto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Bus Stop,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Yoga Studio,Historic Site,Pub,Performing Arts Venue,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Furniture / Home Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Miscellaneous Shop,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,Park


In [172]:
tto_merged.drop(tto_merged[tto_merged['Cluster Labels'].isnull()].index, inplace=True)

In [173]:
tto_merged = tto_merged.astype({'Cluster Labels':int})

### Creating a map for the clusters

In [175]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tto_merged['Latitude'], tto_merged['Longitude'], tto_merged['Neighborhood'], tto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analysis

### Cluster 1 (RED) : Cofee shops and restaurants 

In [176]:
tto_merged.loc[tto_merged['Cluster Labels'] == 0, tto_merged.columns[[1] + list(range(5, tto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Yoga Studio,Historic Site,Pub,Performing Arts Venue,Mexican Restaurant
3,North York,0,Clothing Store,Accessories Store,Furniture / Home Store,Event Space,Vietnamese Restaurant,Coffee Shop,Boutique,Miscellaneous Shop,Dessert Shop,Electronics Store
4,Downtown Toronto,0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,Park
6,Scarborough,0,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
7,North York,0,Gym,Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Sporting Goods Shop
9,Downtown Toronto,0,Café,Tea Room,Clothing Store,Theater,Electronics Store,Burger Joint,Sporting Goods Shop,Japanese Restaurant,Shopping Mall,Sandwich Place
11,Etobicoke,0,Brewery,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
12,Scarborough,0,Bar,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
13,North York,0,Gym,Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Sandwich Place,Sporting Goods Shop
14,East York,0,Pharmacy,Park,Skating Rink,Video Store,Athletics & Sports,Beer Store,Curling Ice,Yoga Studio,Discount Store,Dim Sum Restaurant


### Cluster 2 (PURPLE) : Sports 

In [177]:
tto_merged.loc[tto_merged['Cluster Labels'] == 1, tto_merged.columns[[1] + list(range(5, tto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,1,Baseball Field,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,1,Baseball Field,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


### Cluster 3 (GREEN): Parks

In [178]:
tto_merged.loc[tto_merged['Cluster Labels'] == 2, tto_merged.columns[[1] + list(range(5, tto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Food & Drink Shop,Bus Stop,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
21,York,2,Park,Women's Store,Convenience Store,Bar,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
32,Scarborough,2,Playground,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
35,East York,2,Park,Convenience Store,Intersection,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
61,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
64,York,2,Park,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
66,North York,2,Park,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
85,Scarborough,2,Playground,Park,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
91,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


### Cluster 4 (YELLOW): Pizza!

In [179]:
tto_merged.loc[tto_merged['Cluster Labels'] == 3, tto_merged.columns[[1] + list(range(5, tto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
8,East York,3,Pizza Place,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
10,North York,3,Pizza Place,Pub,Japanese Restaurant,Italian Restaurant,Bakery,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
17,Etobicoke,3,Pizza Place,Beer Store,Coffee Shop,Café,Liquor Store,Pharmacy,Pet Store,Gay Bar,Eastern European Restaurant,Donut Shop
22,Scarborough,3,Coffee Shop,Korean Restaurant,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
50,North York,3,Pizza Place,Italian Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
63,York,3,Pizza Place,Convenience Store,Bus Line,Dessert Shop,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
70,Etobicoke,3,Pizza Place,Coffee Shop,Discount Store,Chinese Restaurant,Middle Eastern Restaurant,Sandwich Place,Intersection,Department Store,Eastern European Restaurant,Drugstore
72,North York,3,Pizza Place,Coffee Shop,Discount Store,Bank,Pharmacy,Grocery Store,Comic Shop,Comfort Food Restaurant,Event Space,Electronics Store
77,Etobicoke,3,Pizza Place,Park,Bus Line,Sandwich Place,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
